Bibliotecas utilizadas

In [1]:
import pandas as pd
import requests as req
import datetime as date
import numpy as np
import math

Funções salvas

In [2]:
def dados_FC():
    response = req.get("https://xivapi.com/freecompany/9234349560946614930?data=FCM")
    return response.json()

def personagens(id):
    response = req.get("https://xivapi.com/character/"+ id)
    return response.json()

def discord():
    headers = {
        'authorization':'MTgxMTQxOTc4NDI2MjQ1MTQx.GWyYGY.tmzxSxYRbfvoS5YlfVl-cfY8vOia3LqiOBYsh8'
    }
    response = req.get(f"https://discord.com/api/v9/channels/766131659111268392/messages", headers=headers)
    return response.json()

def discord_50_m(id):
    headers = {
        'authorization':'MTgxMTQxOTc4NDI2MjQ1MTQx.GWyYGY.tmzxSxYRbfvoS5YlfVl-cfY8vOia3LqiOBYsh8'
    }
    response = req.get(f"https://discord.com/api/v9/channels/766131659111268392/messages?before=" + id + "&limit=50", headers=headers)
    return response.json()
    


4- [FATO] Tabela Membros

In [3]:
FATO_MEMBROS_FC = pd.DataFrame(dados_FC()["FreeCompanyMembers"])
FATO_MEMBROS_FC.drop(["Lang","RankIcon","FeastMatches","Server"],axis = 1, inplace = True)

FATO_MEMBROS_FC["ID"] = FATO_MEMBROS_FC["ID"].astype(str) 
FATO_MEMBROS_FC["Lodestone"] = "https://na.finalfantasyxiv.com/lodestone/character/" + FATO_MEMBROS_FC["ID"] 



5- [REGRAS] Dicionario pra classificação de TNKS, DPS e Healer

Classe de craft descartadas como "N/A"

In [4]:
jobs_dict ={
                "alchemist / alchemist":"N/A",            
                "arcanist / scholar":"HLR",               
                "arcanist / summoner":"DPS",              
                "archer / bard":"DPS",   
                "armorer / armorer":"N/A",
                "astrologian / astrologian":"HLR",        
                "blacksmith / blacksmith":"N/A",          
                "blue mage / blue mage":"DPS",            
                "botanist / botanist":"N/A",              
                "carpenter / carpenter":"N/A",            
                "conjurer / white mage":"HLR",            
                "culinarian / culinarian":"N/A",          
                "dancer / dancer":"DPS", 
                "dark knight / dark knight":"TNK",        
                "fisher / fisher":"N/A", 
                "gladiator / paladin":"TNK",              
                "goldsmith / goldsmith":"N/A",            
                "gunbreaker / gunbreaker":"TNK",          
                "lancer / dragoon":"DPS",
                "leatherworker / leatherworker":"N/A",    
                "machinist / machinist":"DPS",            
                "marauder / warrior":"TNK",               
                "miner / miner":"N/A",   
                "pugilist / monk":"DPS",
                "reaper / reaper":"DPS", 
                "red mage / red mage":"DPS",              
                "rogue / ninja":"DPS",   
                "sage / sage":"HLR",     
                "samurai / samurai":"DPS",
                "thaumaturge / black mage":"DPS",       
                "weaver / weaver":"HLR"      
    
    
            }

3- [BRUTA] Tabela Jobs

In [5]:
RAW_ClassJobs = pd.DataFrame([])
x=0
for x in range(0, FATO_MEMBROS_FC["ID"].count()):
    sup= pd.DataFrame(personagens(FATO_MEMBROS_FC["ID"][x])["Character"]["ClassJobs"])
    sup.drop(list(sup.filter(regex = "Exp")), axis = 1, inplace = True)
    sup.drop(["ClassID","IsSpecialised","JobID","UnlockedState"], axis =1, inplace = True)
    sup["Lodestone"] = "https://na.finalfantasyxiv.com/lodestone/character/" + FATO_MEMBROS_FC["ID"][x]
    RAW_ClassJobs = pd.concat([RAW_ClassJobs,sup])

RAW_ClassJobs.reset_index(drop = True, inplace = True)

2- [ANALITICA] Tabela Jobs

In [6]:
ANALYTICS_90ClassJobs = RAW_ClassJobs

ANALYTICS_90ClassJobs = ANALYTICS_90ClassJobs.replace({"Name":jobs_dict})
ANALYTICS_90ClassJobs["Name"].replace(regex="N/A",value=float("NaN"), inplace= True)
ANALYTICS_90ClassJobs.dropna(subset="Name",inplace= True)

ANALYTICS_90ClassJobs = ANALYTICS_90ClassJobs.query('Level == 90')

sup = list(ANALYTICS_90ClassJobs.columns)
sup[sup.index("Name")] = "Role_Jobs"
ANALYTICS_90ClassJobs.columns = sup

ANALYTICS_90ClassJobs["Qtd_Jobs"] = ANALYTICS_90ClassJobs["Role_Jobs"]
ANALYTICS_90ClassJobs = ANALYTICS_90ClassJobs.groupby(["Lodestone","Role_Jobs"])["Qtd_Jobs"].count().reset_index()

3- [BRUTA] Tabela Discord

In [7]:
RAW_discord=pd.DataFrame(discord())

x = RAW_discord["id"].min() 

while x != "766849288331722803":
    sup = pd.DataFrame(discord_50_m(x))  
    x = sup["id"].min()
    RAW_discord = pd.concat([RAW_discord,sup])

RAW_discord.drop(["type","channel_id","author","attachments","embeds","mentions","mention_roles","pinned","mention_everyone","tts","edited_timestamp","flags","components"], axis =1 , inplace= True)    
RAW_discord["timestamp"] = RAW_discord["timestamp"].astype(str).str[:10]
sup = list(RAW_discord.columns)
sup[sup.index("content")] = "mensagem"
sup[sup.index("timestamp")] = "data"
RAW_discord.columns = sup
RAW_discord.reset_index(drop = True, inplace= True)

2- [ANALITICA] TABELA DISCORD_DIAS_FC

In [8]:
ANALYTICS_discord_part1 = RAW_discord.loc[RAW_discord["mensagem"].str.contains('. Id:', regex=False)]
ANALYTICS_discord_part1 = ANALYTICS_discord_part1.join(ANALYTICS_discord_part1["mensagem"].str.split("(", expand = True))

sup = list(ANALYTICS_discord_part1.columns)
sup[sup.index(0)] = "mensagem_1"
sup[sup.index(1)] = "mensagem_2"
ANALYTICS_discord_part1.columns = sup
ANALYTICS_discord_part1.drop("mensagem", axis =1 , inplace = True)

ANALYTICS_discord_part1 = ANALYTICS_discord_part1.join(ANALYTICS_discord_part1["mensagem_2"].str.split(")", expand = True))

sup = list(ANALYTICS_discord_part1.columns)
sup[sup.index(0)] = "mensagem_3"
sup[sup.index(1)] = "mensagem_4"
ANALYTICS_discord_part1.columns = sup
ANALYTICS_discord_part1.drop(["mensagem_2","mensagem_3"], axis =1 , inplace = True)

ANALYTICS_discord_part1 = ANALYTICS_discord_part1.join(ANALYTICS_discord_part1["mensagem_1"].str.rsplit(" ", n=1, expand = True))

sup = list(ANALYTICS_discord_part1.columns)
sup[sup.index(0)] = "Name"
sup[sup.index(1)] = "mensagem_6"
ANALYTICS_discord_part1.columns = sup
ANALYTICS_discord_part1.drop(["mensagem_1","mensagem_6"], axis =1 , inplace = True)

ANALYTICS_discord_part1 = ANALYTICS_discord_part1.join(ANALYTICS_discord_part1["mensagem_4"].str.rsplit(" ", n=1, expand = True))
sup = list(ANALYTICS_discord_part1.columns)
sup[sup.index(0)] = "Status_entrada_saida"
sup[sup.index(1)] = "id_lodstone"
ANALYTICS_discord_part1.columns = sup
ANALYTICS_discord_part1.drop("mensagem_4", axis =1 , inplace = True)
ANALYTICS_discord_part1["Status_entrada_saida"].replace(regex = "da fc.", value = "", inplace = True)
ANALYTICS_discord_part1["Status_entrada_saida"].replace(regex = "na fc.", value = "", inplace = True)
ANALYTICS_discord_part1["Status_entrada_saida"].replace(regex = " ", value = "", inplace = True)
ANALYTICS_discord_part1["Status_entrada_saida"].replace(regex = "Id:", value = "", inplace = True)



In [10]:

ANALYTICS_discord_part2 = RAW_discord.loc[RAW_discord["mensagem"].str.contains('. Id:', regex=False)==False]
ANALYTICS_discord_part2 = ANALYTICS_discord_part2.join(ANALYTICS_discord_part2["mensagem"].str.split("(", expand = True))

sup = list(ANALYTICS_discord_part2.columns)
sup[sup.index(0)] = "mensagem_1"
sup[sup.index(1)] = "mensagem_2"
ANALYTICS_discord_part2.columns = sup
ANALYTICS_discord_part2.drop("mensagem", axis =1 , inplace = True)

ANALYTICS_discord_part2 = ANALYTICS_discord_part2.join(ANALYTICS_discord_part2["mensagem_2"].str.split(")", expand = True))

sup = list(ANALYTICS_discord_part2.columns)
sup[sup.index(0)] = "mensagem_3"
sup[sup.index(1)] = "Status_entrada_saida"
ANALYTICS_discord_part2.columns = sup
ANALYTICS_discord_part2.drop(["mensagem_2","mensagem_3"], axis =1 , inplace = True)

ANALYTICS_discord_part2 = ANALYTICS_discord_part2.join(ANALYTICS_discord_part2["mensagem_1"].str.rsplit(" ", n=1, expand = True))

sup = list(ANALYTICS_discord_part2.columns)
sup[sup.index(0)] = "Name"
sup[sup.index(1)] = "mensagem_6"
ANALYTICS_discord_part2.columns = sup
ANALYTICS_discord_part2.drop(["mensagem_1","mensagem_6"], axis =1 , inplace = True)


ANALYTICS_discord_part2["Status_entrada_saida"].replace(regex = "da fc.", value = "", inplace = True)
ANALYTICS_discord_part2["Status_entrada_saida"].replace(regex = "na fc.", value = "", inplace = True)
ANALYTICS_discord_part2["Status_entrada_saida"].replace(regex = " ", value = "", inplace = True)
ANALYTICS_discord_part2["id_lodstone"]=" "


In [11]:
ANALYTICS_discord = pd.concat([ANALYTICS_discord_part1,ANALYTICS_discord_part2])

filtro_entrou = ANALYTICS_discord["Status_entrada_saida"] == "entrou"
filtro_saiu = ANALYTICS_discord["Status_entrada_saida"] == "saiu"

In [12]:
ANALYTICS_discord_dias_fc_Ul_entrou = ANALYTICS_discord.loc[filtro_entrou == True].merge(FATO_MEMBROS_FC,how="inner",on = "Name")
ANALYTICS_discord_dias_fc_Ul_entrou["data"] = pd.to_datetime(ANALYTICS_discord_dias_fc_Ul_entrou["data"])
ANALYTICS_discord_dias_fc_Ul_entrou = ANALYTICS_discord_dias_fc_Ul_entrou.groupby(["id_lodstone","Name"])["data"].max().reset_index()
ANALYTICS_discord_dias_fc_Ul_entrou["data"] = ANALYTICS_discord_dias_fc_Ul_entrou['data'].astype(str).str[:10]
x=0
sup = pd.Series([], dtype="object")
for x in range(x, ANALYTICS_discord_dias_fc_Ul_entrou["data"].count()):
    sup[x] = (date.datetime.today() - date.datetime.strptime(ANALYTICS_discord_dias_fc_Ul_entrou["data"][x],'%Y-%m-%d'))
ANALYTICS_discord_dias_fc_Ul_entrou["Qtd_dias"] = sup.dt.components.days

ANALYTICS_discord_dias_fc_Ul_entrou["Qtd_dias"] = ANALYTICS_discord_dias_fc_Ul_entrou["Qtd_dias"].astype(float)



2- [ANALITICA] CARGOS FC

In [13]:
ANALYTICS_Cargos = FATO_MEMBROS_FC.merge(ANALYTICS_discord_dias_fc_Ul_entrou, how="left", on= "Name")
ANALYTICS_Cargos = pd.merge(ANALYTICS_Cargos,ANALYTICS_discord_dias_fc_Ul_entrou, how="left", left_on= ["ID"], right_on= ["id_lodstone"])
ANALYTICS_Cargos = ANALYTICS_Cargos.merge(ANALYTICS_90ClassJobs, how="left", on= "Lodestone")
ANALYTICS_Cargos["Qtd_dias"] = float("NaN")
for x in range(0,ANALYTICS_Cargos["ID"].count()):
    if math.isnan(ANALYTICS_Cargos["Qtd_dias_y"][x]):
        ANALYTICS_Cargos["Qtd_dias"][x] = ANALYTICS_Cargos["Qtd_dias_x"][x]
    else:
        ANALYTICS_Cargos["Qtd_dias"][x] = ANALYTICS_Cargos["Qtd_dias_y"][x]

ANALYTICS_Cargos = ANALYTICS_Cargos.groupby(["ID","Avatar","Name_x","Rank"]).agg({"Qtd_dias":"mean", "Role_Jobs":"count","Qtd_Jobs":"sum"}).reset_index()

sup = list(ANALYTICS_Cargos.columns)
sup[sup.index("Name_x")] = "Name"
ANALYTICS_Cargos.columns = sup



C:\Users\35383\AppData\Local\Temp\ipykernel_25796\170596366.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ANALYTICS_Cargos["Qtd_dias"][x] = ANALYTICS_Cargos["Qtd_dias_x"][x]
C:\Users\35383\AppData\Local\Temp\ipykernel_25796\170596366.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ANALYTICS_Cargos["Qtd_dias"][x] = ANALYTICS_Cargos["Qtd_dias_y"][x]


6- [PARTICULARIDADE] Tratamento para pessoas que mudaram de Nick

In [14]:
ANALYTICS_Cargos["Qtd_dias"] = ANALYTICS_Cargos["Qtd_dias"].astype(float)

x=0
for x in range(0,ANALYTICS_Cargos["ID"].count()):
    if math.isnan(ANALYTICS_Cargos["Qtd_dias"][x]):
        ANALYTICS_Cargos["Qtd_dias"][x] = 0




C:\Users\35383\AppData\Local\Temp\ipykernel_25796\2516148593.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ANALYTICS_Cargos["Qtd_dias"][x] = 0


5- [REGRAS] CARGO DA FC

In [15]:
filtro_cargos_sapling = ANALYTICS_Cargos["Qtd_dias"]<90
filtro_cargos_senior_tree = np.logical_and(np.logical_and(ANALYTICS_Cargos["Qtd_dias"]>=90, ANALYTICS_Cargos["Qtd_dias"]<180), ANALYTICS_Cargos["Role_Jobs"]>0 )
filtro_cargos_lunar_tree = np.logical_and(ANALYTICS_Cargos["Qtd_dias"]>=180, ANALYTICS_Cargos["Role_Jobs"]==3)

1- [PRODUCAO] TABELA DE CARGOS A SEREM ATUALIZADOS

In [16]:
PROD_Cargos = ANALYTICS_Cargos
PROD_Cargos["Rank_recomendado"] = float("Nan")

PROD_Cargos["Rank_recomendado"].loc[filtro_cargos_sapling ==True] = "Sapling"
PROD_Cargos["Rank_recomendado"].loc[filtro_cargos_senior_tree ==True] = "Senior Tree"
PROD_Cargos["Rank_recomendado"].loc[filtro_cargos_lunar_tree ==True] = "Lunar Tree"
PROD_Cargos = PROD_Cargos.loc[PROD_Cargos["Rank"] != PROD_Cargos["Rank_recomendado"]]
PROD_Cargos = PROD_Cargos.loc[PROD_Cargos["Qtd_dias"]>0]


PROD_Cargos["Rank"].replace("1st Div - Mod",float("NaN"),inplace=True)
PROD_Cargos["Rank"].replace("2nd Div - Tech",float("NaN"),inplace=True)
PROD_Cargos["Rank"].replace("3rd Div - Estr",float("NaN"),inplace=True)
PROD_Cargos["Rank"].replace("Counsel Master",float("NaN"),inplace=True)
PROD_Cargos["Rank"].replace("Lunar Tree",float("NaN"),inplace=True)
PROD_Cargos = PROD_Cargos.dropna(subset=["Rank_recomendado","Rank"])
sup = list(PROD_Cargos.columns)
sup[sup.index("Role_Jobs")] = "Qtds_role_jobs_lvl_90"
PROD_Cargos.columns = sup

PROD_Cargos = PROD_Cargos.groupby(["ID","Avatar","Name","Rank","Rank_recomendado"]).agg({"Qtd_dias":"mean","Qtds_role_jobs_lvl_90":"mean","Qtd_Jobs":"mean"}).reset_index()


PROD_Cargos.set_index("ID", inplace=True)

PROD_Cargos.to_csv("ffxiv_cargos_fc.csv")

C:\Users\35383\AppData\Local\Temp\ipykernel_25796\2642456434.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PROD_Cargos["Rank_recomendado"].loc[filtro_cargos_sapling ==True] = "Sapling"
